In [1]:
import os
import re
import math
import string
import codecs
import json
from itertools import product
from inspect import getsourcefile
from io import open

In [7]:
#boost word가 감정 강도를 바꾸는 정도 (이론적으로 도출된 결과 값이라고 설명되어 있음) 
#boost word : ex) 매우, 극도로, 아주, 약간 등등 수식하는 단어의 강도를 나타내는 부사적 어휘
#이 수치를 그대로 사용할 것인가...?
B_INCR = 0.293
B_DECR = -0.293

#영어의 경우 대문자로 강조 할 수 있음, 이떄 감정 강도를 바꾸는 정도 
#한글 버전의 경우 불필요한 변수로 생각됨
#
C_INCR = 0.733
N_SCALAR = -0.74

#부정적 의미의 동사 형용사 부사 등의 보조어를 모아둔 리스트 (len = 59)
#한글의 경우 어떤 용어로 대채할 지 조사 필요함 

NEGATE = \
    ["aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
     "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
     "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
     "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
     "neednt", "needn't", "never", "none", "nope", "nor", "not", "nothing", "nowhere",
     "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
     "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
     "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite"]


#BOOSTER WORD가 감정강도에 긍정, 부정인지 정리한 DICT (len = 84)
BOOSTER_DICT = \
    {"absolutely": B_INCR, "amazingly": B_INCR, "awfully": B_INCR, 
     "completely": B_INCR, "considerable": B_INCR, "considerably": B_INCR,
     "decidedly": B_INCR, "deeply": B_INCR, "effing": B_INCR, "enormous": B_INCR, "enormously": B_INCR,
     "entirely": B_INCR, "especially": B_INCR, "exceptional": B_INCR, "exceptionally": B_INCR, 
     "extreme": B_INCR, "extremely": B_INCR,
     "fabulously": B_INCR, "flipping": B_INCR, "flippin": B_INCR, "frackin": B_INCR, "fracking": B_INCR,
     "fricking": B_INCR, "frickin": B_INCR, "frigging": B_INCR, "friggin": B_INCR, "fully": B_INCR, 
     "fuckin": B_INCR, "fucking": B_INCR, "fuggin": B_INCR, "fugging": B_INCR,
     "greatly": B_INCR, "hella": B_INCR, "highly": B_INCR, "hugely": B_INCR, 
     "incredible": B_INCR, "incredibly": B_INCR, "intensely": B_INCR, 
     "major": B_INCR, "majorly": B_INCR, "more": B_INCR, "most": B_INCR, "particularly": B_INCR,
     "purely": B_INCR, "quite": B_INCR, "really": B_INCR, "remarkably": B_INCR,
     "so": B_INCR, "substantially": B_INCR,
     "thoroughly": B_INCR, "total": B_INCR, "totally": B_INCR, "tremendous": B_INCR, "tremendously": B_INCR,
     "uber": B_INCR, "unbelievably": B_INCR, "unusually": B_INCR, "utter": B_INCR, "utterly": B_INCR,
     "very": B_INCR,
     "almost": B_DECR, "barely": B_DECR, "hardly": B_DECR, "just enough": B_DECR,
     "kind of": B_DECR, "kinda": B_DECR, "kindof": B_DECR, "kind-of": B_DECR,
     "less": B_DECR, "little": B_DECR, "marginal": B_DECR, "marginally": B_DECR,
     "occasional": B_DECR, "occasionally": B_DECR, "partly": B_DECR,
     "scarce": B_DECR, "scarcely": B_DECR, "slight": B_DECR, "slightly": B_DECR, "somewhat": B_DECR,
     "sort of": B_DECR, "sorta": B_DECR, "sortof": B_DECR, "sort-of": B_DECR}

#lexcicon에 포함되지 않은 감성적인 관용적 표현들 모음
#본 코드에서는 아직 구현하지 않음.
SENTIMENT_LADEN_IDIOMS = {"cut the mustard": 2, "hand to mouth": -2,
                          "back handed": -2, "blow smoke": -2, "blowing smoke": -2,
                          "upper hand": 1, "break a leg": 2,
                          "cooking with gas": 2, "in the black": 2, "in the red": -2,
                          "on the ball": 2, "under the weather": -2}

#Lexicon에 포함된 특이 케이스의 관용적 표현들이라는데 무슨 기준인지 잘 모르겠슴....
SPECIAL_CASES = {"the shit": 3, "the bomb": 3, "bad ass": 1.5, "badass": 1.5, "bus stop": 0.0,
                 "yeah right": -2, "kiss of death": -1.5, "to die for": 3, 
                 "beating heart": 3.1, "broken heart": -2.9 }



In [8]:
'''Static methods 구현.
class 객체를 만들지 않아도, 모듈을 import하기만 해도
사용할 수 있는 함수들을 뜻함
즉, class 선언 외부에 존재하는 함수들임

(class 내부에 선언된 함수의 경우 객체를 만들어야 사용가능하고 non-static methods라고 함)
'''

def negated(input_words, include_nt=True):
    """
    input_words에 NEGATE(부정형 보조어 리스트)에 속하는 word가 존재 : return True
    input_words에 NEGATE(부정형 보조어 리스트)에 속하는 word가 존재 x : return False
    
    include_nt = True 일때  "n't"를 포함하는 단어가 포함되면 retrun True
    """
    input_words = [str(w).lower() for w in input_words]
    neg_words = []
    neg_words.extend(NEGATE)
    for word in neg_words:
        if word in input_words:
            return True
    if include_nt:
        for word in input_words:
            if "n't" in word:
                return True
    '''원래 주석처리 되어 있던 부분. "at least"를 부정형 보조어로 처리할지 결정하지 못한 듯 보임.
        if "least" in input_words:
        i = input_words.index("least")
        if i > 0 and input_words[i - 1] != "at":
            return True'''
    return False


def normalize(score, alpha=15):
    """
    score를 정규화 시켜주는 함수
    """
    norm_score = score / math.sqrt((score * score) + alpha)
    if norm_score < -1.0:
        return -1.0
    elif norm_score > 1.0:
        return 1.0
    else:
        return norm_score
    


def allcap_differential(words):
    """
    words에 포함된 일부의 word가 전부 대문자로 표기된 경우가 있으면 return True
    한글 버전에서는 구현이 필요 없을 것 같다.
    """
    is_different = False
    allcap_words = 0
    for word in words:
        if word.isupper():
            allcap_words += 1
    cap_differential = len(words) - allcap_words
    if 0 < cap_differential < len(words):
        is_different = True
    return is_different


def scalar_inc_dec(word, valence, is_cap_diff):
    """
    valence : word 뒤에 오는 단어의 감성 수치
    scalar : word가 BOOSTER_DICT에 포함된 단어일 경우 B_INCR, 또는 B_DECR의 값을 가짐
    
    """
    scalar = 0.0
    word_lower = word.lower()
    if word_lower in BOOSTER_DICT:
        scalar = BOOSTER_DICT[word_lower]
        if valence < 0:
            scalar *= -1
        # word가 대문자인지 여부에 따라 강도 변경하는 부분.
        # 한글 버전에서는 불필요함
        if word.isupper() and is_cap_diff:
            if valence > 0:
                scalar += C_INCR
            else:
                scalar -= C_INCR
    return scalar

In [ ]:
class SentiText(object):
    """
    객체 입력값 : emoji를 text로 변환시킨 text 문자열
    
    
    words_and_emoticons : 구둣점을 제외한 text
    is_cap_diff : 대문자 강조 표현이 사용되었는지 여부
    text : 처음 입력된 문자열
    
    """

    def __init__(self, text):
        if not isinstance(text, str):
            text = str(text).encode('utf-8')                              #text가 srt이 아니면 str로 변환해준다.
        self.text = text
        self.words_and_emoticons = self._words_and_emoticons()            # 구둣점을 제거하고 words와 emoticons을 남긴 text를 word_and_emoticons에 저장
        # doesn't separate words from\
        # adjacent punctuation (keeps emoticons & contractions)
        self.is_cap_diff = allcap_differential(self.words_and_emoticons)  #대문자 강조 표현이 있으면 True 아니면 False, 한글 구현에서는 불필요

    @staticmethod
    def _strip_punc_if_word(token):
        """
        구두점 제거할 때 이모티콘인지 판별하는 함수
        아래의 _words_and_emoticons 함수에서 사용함
        """
        stripped = token.strip(string.punctuation)
        if len(stripped) <= 2:
            return token
        return stripped

    def _words_and_emoticons(self):
        """
        이모티콘을 제외한 선행 후행 구둣점 제거
        """
        wes = self.text.split()
        stripped = list(map(self._strip_punc_if_word, wes))
        return stripped

In [ ]:
class SentimentIntensityAnalyzer(object):
    """
    Give a sentiment intensity score to sentences.
    """
    
    
    def __init__(self, lexicon_file="vader_lexicon.txt", emoji_lexicon="emoji_utf8_lexicon.txt"):
        '''
        lexicon_file : 단어 감성사전 => 한글버전 필요함.
        emoji_lexicon : 이모티콘 감성사전 => 그대로 사용할지 말지 판단 필요함.
        '''
        _this_module_file_path_ = os.path.abspath(getsourcefile(lambda: 0)) #현재 모듈이 실행중인 파일의 절대경로 저장
        lexicon_full_filepath = os.path.join(os.path.dirname(_this_module_file_path_), lexicon_file) #lexicon_file 경로 저장
        with codecs.open(lexicon_full_filepath, encoding='utf-8') as f:
            self.lexicon_full_filepath = f.read()
        self.lexicon = self.make_lex_dict()

        emoji_full_filepath = os.path.join(os.path.dirname(_this_module_file_path_), emoji_lexicon) #emoji_lexicon 경로 저장
        with codecs.open(emoji_full_filepath, encoding='utf-8') as f:
            self.emoji_full_filepath = f.read()
        self.emojis = self.make_emoji_dict()
        
        ''' 결과
        self.lexicon  : lexcion dictionary
        self.emojis : emojis dictionary
        '''
    
    def make_lex_dict(self):
        """
        lexicion파일을 dictionary 형태로 바꿔준다. (__init__에서 사용됨)
        
        return:
        lex_dict = {word : (float(감성점수))}
        """
        lex_dict = {}
        for line in self.lexicon_full_filepath.rstrip('\n').split('\n'):
            if not line:
                continue
            (word, measure) = line.strip().split('\t')[0:2]
            lex_dict[word] = float(measure)
        return lex_dict
        
    def make_emoji_dict(self):
        """
        emojis lexicon파일을 dictionary 형태로 바꿔준다.(__init__에서 사용됨)
        
        return:
        emoji_dict = {emoji : emoji의 의미(text)}
        """
        emoji_dict = {}
        for line in self.emoji_full_filepath.rstrip('\n').split('\n'):
            (emoji, description) = line.strip().split('\t')[0:2]
            emoji_dict[emoji] = description
        return emoji_dict
    
    def polarity_scores(self, text):
        """
        text기준으로 감정 강도를 float 점수로 반환해줌.
        
        """
        # emoji를 텍스트로 변환하는 작업
        text_no_emoji = ""
        prev_space = True
        for chr in text:
            if chr in self.emojis:              #text의 문자가 emoji일 경우
                description = self.emojis[chr]  #description에 emoji 뜻 저장
                if not prev_space:              #emoji 앞에 여백(space)가 없으면
                    text_no_emoji += ' '        #text_no_emoji에 여백을 만들어주고
                text_no_emoji += description    #text_no_emoji에 decription 붙여준다.
                prev_space = False
            else:                               #emoji가 아닐 경우
                text_no_emoji += chr            #text_no_emoji에 chr 추가
                prev_space = chr == ' '         #문자가 공백으로 입력된 경우 prev_space == True로 바꾸어줌
        text = text_no_emoji.strip()            #최종 결과 text_no_emoji의 공백 제거 후 text에 다시 저장

        sentitext = SentiText(text)             # 구둣점 제거한 text, 대문자 강조 사용 여부를 저장하고 있는 객체 생성

        sentiments = []
        words_and_emoticons = sentitext.words_and_emoticons
        for i, item in enumerate(words_and_emoticons):
            
            valence = 0                         #valence 감성 수치(점수)
            
            if item.lower() in BOOSTER_DICT:    #item이 BOOSTER_DICT에 있으면 0
                sentiments.append(valence)
                continue
            if (i < len(words_and_emoticons) - 1 and item.lower() == "kind" and  #현재 item이 "kind"이고 다음 item이 "of"이면 0
                    words_and_emoticons[i + 1].lower() == "of"):                 #아마 "kind"가 친철한 이라는 의미도 가지고 있어서 넣어놓은 알고리즘인 듯
                sentiments.append(valence)
                continue

            sentiments = self.sentiment_valence(valence, sentitext, item, i, sentiments) #위의 조건에 해당하지 않으면 snetiment_valence 함수를 사용하여 valence 값 넣기

        sentiments = self._but_check(words_and_emoticons, sentiments) #문장에 "but"이 있는지 확인하고 전체적인 valence 조정

        valence_dict = self.score_valence(sentiments, text) #각각의 단어로부터 구한 valence를 저장한 리스트 sentiments로부터 최종적인 문장의 감성 강도 계산

        return valence_dict #valence_dict는 neg, neu, pos, compound 값을 가지고 있음
    
    def sentiment_valence(self, valence, sentitext, item, i, sentiments):
        '''
        valence : 감성 수치 변수
        sentitext : 구둣점 제거한 text, 대문자 강조 사용 여부를 저장하고 있는 객체
        item : valence를 계산할 단어
        i : text에서 item의 index
        sentiments : valence를 추가하여 return할 list
        
        return
        : sentiments
        '''
        is_cap_diff = sentitext.is_cap_diff    #대문자 강조 표현이 text에 있는지 확인                                                                   
        words_and_emoticons = sentitext.words_and_emoticons 
        item_lowercase = item.lower()
        if item_lowercase in self.lexicon:
            valence = self.lexicon[item_lowercase]
                
            # 수식하는 용도의 "no" 와 독립 어휘로서의 "no"를 체크하는 부분
            if item_lowercase == "no" and i != len(words_and_emoticons)-1 and words_and_emoticons[i + 1].lower() in self.lexicon: # item이 "no"이고 마지막 단어가 아닐떄, 그리고 다음 item이 lexicon에 있을경우
                # "no"를 lexicon의 valence로 사용하지 않음(수식어로 체크)
                valence = 0.0
            #현재 item 앞에 "no"가 있으면 N_SCALAR를 valence에 곱한다.
            if (i > 0 and words_and_emoticons[i - 1].lower() == "no") \
               or (i > 1 and words_and_emoticons[i - 2].lower() == "no") \
               or (i > 2 and words_and_emoticons[i - 3].lower() == "no" and words_and_emoticons[i - 1].lower() in ["or", "nor"] ):
                valence = self.lexicon[item_lowercase] * N_SCALAR
            
            # 대문자 강조 표현에 valence 조정 (한글 구현에 필요 x)
            if item.isupper() and is_cap_diff:
                if valence > 0:
                    valence += C_INCR
                else:
                    valence -= C_INCR
            
            for start_i in range(0, 3):
                # 현재 item 앞에 온 booster_dict이 존재하는지, 거리가 얼마나 되는지에 따라 valence 조정한다. 
                if i > start_i and words_and_emoticons[i - (start_i + 1)].lower() not in self.lexicon:
                    s = scalar_inc_dec(words_and_emoticons[i - (start_i + 1)], valence, is_cap_diff) #s : valence를 조정할 scalar 값
                    if start_i == 1 and s != 0:  #item과 booster_word와의 거리에 따른 스칼라값 조정
                        s = s * 0.95
                    if start_i == 2 and s != 0:
                        s = s * 0.9
                    valence = valence + s       #최종적으로 구해진 scalar값으로 valence 조정
                    valence = self._negation_check(valence, words_and_emoticons, start_i, i) #부정문인지 체크하여 valence 조정
                    if start_i == 2:
                        valence = self._special_idioms_check(valence, words_and_emoticons, i)

            valence = self.least_check(valence, words_and_emoticons, i) #item 앞에 단어가 "least"인지에 따라 valence 조정
        sentiments.append(valence)
        return sentiments

    def _least_check(self, valence, words_and_emoticons, i):
        # check for negation case using "least"
        if i > 1 and words_and_emoticons[i - 1].lower() not in self.lexicon \
                and words_and_emoticons[i - 1].lower() == "least":
            if words_and_emoticons[i - 2].lower() != "at" and words_and_emoticons[i - 2].lower() != "very":
                valence = valence * N_SCALAR
        elif i > 0 and words_and_emoticons[i - 1].lower() not in self.lexicon \
                and words_and_emoticons[i - 1].lower() == "least":
            valence = valence * N_SCALAR
        return valence

    @staticmethod
    def _but_check(words_and_emoticons, sentiments):
        # check for modification in sentiment due to contrastive conjunction 'but'
        words_and_emoticons_lower = [str(w).lower() for w in words_and_emoticons]
        if 'but' in words_and_emoticons_lower:
            bi = words_and_emoticons_lower.index('but')
            for sentiment in sentiments:
                si = sentiments.index(sentiment)
                if si < bi:
                    sentiments.pop(si)
                    sentiments.insert(si, sentiment * 0.5)
                elif si > bi:
                    sentiments.pop(si)
                    sentiments.insert(si, sentiment * 1.5)
        return sentiments

    @staticmethod
    def _special_idioms_check(valence, words_and_emoticons, i):
        words_and_emoticons_lower = [str(w).lower() for w in words_and_emoticons]
        onezero = "{0} {1}".format(words_and_emoticons_lower[i - 1], words_and_emoticons_lower[i])

        twoonezero = "{0} {1} {2}".format(words_and_emoticons_lower[i - 2],
                                          words_and_emoticons_lower[i - 1], words_and_emoticons_lower[i])

        twoone = "{0} {1}".format(words_and_emoticons_lower[i - 2], words_and_emoticons_lower[i - 1])

        threetwoone = "{0} {1} {2}".format(words_and_emoticons_lower[i - 3],
                                           words_and_emoticons_lower[i - 2], words_and_emoticons_lower[i - 1])

        threetwo = "{0} {1}".format(words_and_emoticons_lower[i - 3], words_and_emoticons_lower[i - 2])

        sequences = [onezero, twoonezero, twoone, threetwoone, threetwo]

        for seq in sequences:
            if seq in SPECIAL_CASES:
                valence = SPECIAL_CASES[seq]
                break

        if len(words_and_emoticons_lower) - 1 > i:
            zeroone = "{0} {1}".format(words_and_emoticons_lower[i], words_and_emoticons_lower[i + 1])
            if zeroone in SPECIAL_CASES:
                valence = SPECIAL_CASES[zeroone]
        if len(words_and_emoticons_lower) - 1 > i + 1:
            zeroonetwo = "{0} {1} {2}".format(words_and_emoticons_lower[i], words_and_emoticons_lower[i + 1],
                                              words_and_emoticons_lower[i + 2])
            if zeroonetwo in SPECIAL_CASES:
                valence = SPECIAL_CASES[zeroonetwo]

        # check for booster/dampener bi-grams such as 'sort of' or 'kind of'
        n_grams = [threetwoone, threetwo, twoone]
        for n_gram in n_grams:
            if n_gram in BOOSTER_DICT:
                valence = valence + BOOSTER_DICT[n_gram]
        return valence

    @staticmethod
    def _sentiment_laden_idioms_check(valence, senti_text_lower):
        # Future Work
        # check for sentiment laden idioms that don't contain a lexicon word
        idioms_valences = []
        for idiom in SENTIMENT_LADEN_IDIOMS:
            if idiom in senti_text_lower:
                print(idiom, senti_text_lower)
                valence = SENTIMENT_LADEN_IDIOMS[idiom]
                idioms_valences.append(valence)
        if len(idioms_valences) > 0:
            valence = sum(idioms_valences) / float(len(idioms_valences))
        return valence

    @staticmethod
    def _negation_check(valence, words_and_emoticons, start_i, i):
        words_and_emoticons_lower = [str(w).lower() for w in words_and_emoticons]
        if start_i == 0:
            if negated([words_and_emoticons_lower[i - (start_i + 1)]]):  # 1 word preceding lexicon word (w/o stopwords)
                valence = valence * N_SCALAR
        if start_i == 1:
            if words_and_emoticons_lower[i - 2] == "never" and \
                    (words_and_emoticons_lower[i - 1] == "so" or
                     words_and_emoticons_lower[i - 1] == "this"):
                valence = valence * 1.25
            elif words_and_emoticons_lower[i - 2] == "without" and \
                    words_and_emoticons_lower[i - 1] == "doubt":
                valence = valence
            elif negated([words_and_emoticons_lower[i - (start_i + 1)]]):  # 2 words preceding the lexicon word position
                valence = valence * N_SCALAR
        if start_i == 2:
            if words_and_emoticons_lower[i - 3] == "never" and \
                    (words_and_emoticons_lower[i - 2] == "so" or words_and_emoticons_lower[i - 2] == "this") or \
                    (words_and_emoticons_lower[i - 1] == "so" or words_and_emoticons_lower[i - 1] == "this"):
                valence = valence * 1.25
            elif words_and_emoticons_lower[i - 3] == "without" and \
                    (words_and_emoticons_lower[i - 2] == "doubt" or words_and_emoticons_lower[i - 1] == "doubt"):
                valence = valence
            elif negated([words_and_emoticons_lower[i - (start_i + 1)]]):  # 3 words preceding the lexicon word position
                valence = valence * N_SCALAR
        return valence

    def _punctuation_emphasis(self, text):
        # !,? 갯수 (최댓값 4)에 따라 punct_emph_amplifier 값 반환
        ep_amplifier = self._amplify_ep(text) # ! 개수 따른 값
        qm_amplifier = self._amplify_qm(text) # ? 개수 따른 값
        punct_emph_amplifier = ep_amplifier + qm_amplifier
        return punct_emph_amplifier

    @staticmethod
    def _amplify_ep(text):
        # check for added emphasis resulting from exclamation points (up to 4 of them)
        ep_count = text.count("!")
        if ep_count > 4:
            ep_count = 4
        # (empirically derived mean sentiment intensity rating increase for
        # exclamation points)
        ep_amplifier = ep_count * 0.292
        return ep_amplifier

    @staticmethod
    def _amplify_qm(text):
        # check for added emphasis resulting from question marks (2 or 3+)
        qm_count = text.count("?")
        qm_amplifier = 0
        if qm_count > 1:
            if qm_count <= 3:
                # (empirically derived mean sentiment intensity rating increase for
                # question marks)
                qm_amplifier = qm_count * 0.18
            else:
                qm_amplifier = 0.96
        return qm_amplifier

    @staticmethod
    def _sift_sentiment_scores(sentiments):
        """
        pos_sum : sentiments에서 0보다 큰 요소들의 합
        neg_sum : sentiments에서 0보다 작은 요소들의 합
        neu_count : 요소의 값이 0인 것 의 갯수
        """
        pos_sum = 0.0
        neg_sum = 0.0
        neu_count = 0
        for sentiment_score in sentiments:
            if sentiment_score > 0:
                pos_sum += (float(sentiment_score) + 1)  # compensates for neutral words that are counted as 1
            if sentiment_score < 0:
                neg_sum += (float(sentiment_score) - 1)  # when used with math.fabs(), compensates for neutrals
            if sentiment_score == 0:
                neu_count += 1
        return pos_sum, neg_sum, neu_count

    def score_valence(self, sentiments, text):
        if sentiments:
            sum_s = float(sum(sentiments))
            # !,?에 따른 valence 조정값
            punct_emph_amplifier = self._punctuation_emphasis(text)
            if sum_s > 0:
                sum_s += punct_emph_amplifier #긍정 문장이면 더하고
            elif sum_s < 0:
                sum_s -= punct_emph_amplifier #부정 문장이면 뺀다.

            compound = normalize(sum_s) # sum_s 를 최대 1, 최소 -1로 정규화
            # discriminate between positive, negative and neutral sentiment scores
            pos_sum, neg_sum, neu_count = self._sift_sentiment_scores(sentiments)
            
            #pos_sum과 neg_sum의 절대값을 비교
            if pos_sum > math.fabs(neg_sum):   #pos_sum이 크면 pos_sum에 ?!강조에 따른 valence 조정 더함
                pos_sum += punct_emph_amplifier
            elif pos_sum < math.fabs(neg_sum): #neg_sum이 크면 neg_sum에 ?!강조에 따른 valence 조정 뺌
                neg_sum -= punct_emph_amplifier

            total = pos_sum + math.fabs(neg_sum) + neu_count
            pos = math.fabs(pos_sum / total)
            neg = math.fabs(neg_sum / total)
            neu = math.fabs(neu_count / total)

        else:
            compound = 0.0
            pos = 0.0
            neg = 0.0
            neu = 0.0

        sentiment_dict = \
            {"neg": round(neg, 3),
             "neu": round(neu, 3),
             "pos": round(pos, 3),
             "compound": round(compound, 4)}

        return sentiment_dict

